In [12]:
import warnings
warnings.filterwarnings("ignore")

from carla.data.catalog import OnlineCatalog
from carla import MLModelCatalog
from carla.recourse_methods import Clue, Wachter
from carla.models.negative_instances import predict_negative_instances
from carla.evaluation.benchmark import Benchmark
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
from time import perf_counter

num = 10
data_name = "compas"

In [2]:
dataset = OnlineCatalog(data_name)

In [6]:
model = MLModelCatalog(dataset, "ann", backend="pytorch")
model.train(
    learning_rate = 0.001,
    epochs = 10,
    max_depth = 50,
    n_estimators = 50,
    batch_size = 20,
    force_train = True
)

balance on test set 0.8131345863037374, balance on test set 0.8191834089436163
Epoch 0/9
----------
train Loss: 0.5864 Acc: 0.8015

test Loss: 0.5370 Acc: 0.8192

Epoch 1/9
----------
train Loss: 0.5190 Acc: 0.8131

test Loss: 0.4964 Acc: 0.8192

Epoch 2/9
----------
train Loss: 0.4815 Acc: 0.8131

test Loss: 0.4501 Acc: 0.8192

Epoch 3/9
----------
train Loss: 0.4432 Acc: 0.8181

test Loss: 0.4199 Acc: 0.8451

Epoch 4/9
----------
train Loss: 0.4173 Acc: 0.8378

test Loss: 0.3962 Acc: 0.8406

Epoch 5/9
----------
train Loss: 0.4011 Acc: 0.8386

test Loss: 0.3796 Acc: 0.8451

Epoch 6/9
----------
train Loss: 0.3896 Acc: 0.8395

test Loss: 0.3697 Acc: 0.8509

Epoch 7/9
----------
train Loss: 0.3824 Acc: 0.8414

test Loss: 0.3617 Acc: 0.8471

Epoch 8/9
----------
train Loss: 0.3771 Acc: 0.8449

test Loss: 0.3699 Acc: 0.8561

Epoch 9/9
----------
train Loss: 0.3738 Acc: 0.8438

test Loss: 0.3596 Acc: 0.8542



In [7]:
hyperparams = {
    "data_name": data_name,
    "train_vae": True,
    "width": 10,
    "depth": 3,
    "latent_dim": 12,
    "batch_size": 64,
    "epochs": 1,
    "lr": 0.001,
    "early_stop": 20,
}
# load a recourse model and pass black box model
cl = Clue(dataset, model, hyperparams)

wa = Wachter(model, {"loss_type": "BCE"})

[INFO] 
Net: [utils.py __init__]
[INFO] VAE_gauss_net [fc_gauss_cat.py __init__]
[INFO] Total params: 0.00M [fc_gauss_cat.py create_net]
[INFO] 
Network: [train.py train_VAE]
[INFO] 
Train: [train.py train_VAE]
[INFO] init cost variables: [train.py train_VAE]
[INFO] it 0/1, vlb -11.090858,  [train.py train_VAE]
[INFO] time: 0.607994 seconds
 [train.py train_VAE]
[INFO] vlb -7.911866 (-inf)
 [train.py train_VAE]
[INFO] Writting C:\Users\drobi\carla\models\autoencoders\clue\fc_VAE_compas_models\theta_best.dat
 [utils.py save]
[INFO] Writting C:\Users\drobi\carla\models\autoencoders\clue\fc_VAE_compas_models/theta_last.dat
 [utils.py save]
[INFO] average time: 0.698997 seconds
 [train.py train_VAE]
[INFO] 
RESULTS: [train.py train_VAE]
[INFO] best_vlb_dev: -7.911866 [train.py train_VAE]
[INFO] best_vlb_train: -11.090858 [train.py train_VAE]
[INFO] nb_parameters: 1111 (1.08KB)
 [train.py train_VAE]
[INFO] 
Net: [utils.py __init__]
[INFO] VAE_gauss_net [fc_gauss_cat.py __init__]
[INFO] Tota

In [17]:
factuals = predict_negative_instances(model, dataset._df).sample(10)

wa_counterfactuals = wa.get_counterfactuals(factuals)
cl_counterfactuals = cl.get_counterfactuals(factuals)

[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]


TypeError: 'builtin_function_or_method' object is not iterable

In [ ]:
display(factuals.index)
display(cl_counterfactuals.index)
display(wa_counterfactuals.index)

In [26]:
benchmark = Benchmark(model, wa, factuals)

[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]


In [27]:
benchmark.run_benchmark()

,Distance_1,Distance_2,Distance_3,Distance_4,Constraint_Violation,Redundancy,y-Nearest-Neighbours,Success_Rate,Average_Time
0,4.0,0.642310,0.103156,0.161960,1,1,0.78,1.0,0.022339
1,4.0,0.279089,0.019474,0.070342,1,2,NaN,NaN,NaN
2,4.0,0.199984,0.009998,0.050092,1,1,NaN,NaN,NaN
3,4.0,0.239277,0.014314,0.060007,1,2,NaN,NaN,NaN
4,4.0,0.040000,0.000400,0.010000,1,6,NaN,NaN,NaN
5,4.0,0.317745,0.025241,0.079659,1,1,NaN,NaN,NaN
6,4.0,0.239206,0.014305,0.059878,1,3,NaN,NaN,NaN
7,4.0,0.079986,0.001599,0.020006,1,2,NaN,NaN,NaN
8,4.0,0.119796,0.003588,0.029961,1,1,NaN,NaN,NaN
9,4.0,0.159541,0.006363,0.039921,1,3,NaN,NaN,NaN


In [ ]:
factuals.index
model.feature_input_order

In [22]:
df_cfs = pd.DataFrame(
    np.array(factuals[model.feature_input_order]), columns=model.feature_input_order, index=list(factuals.index)
)

In [24]:
df_cfs.index = factuals.index

In [25]:
df_cfs

,age,two_year_recid,priors_count,length_of_stay,c_charge_degree_M,race_Other,sex_Male
4273,0.358974,1.0,0.684211,0.21000,0.0,1.0,1.0
3443,0.166667,0.0,0.394737,0.13125,0.0,0.0,1.0
1572,0.076923,1.0,0.289474,0.00125,0.0,1.0,1.0
381,0.115385,1.0,0.342105,0.00250,1.0,0.0,1.0
2024,0.076923,1.0,0.105263,0.00250,0.0,0.0,1.0
3623,0.230769,1.0,0.421053,0.01125,0.0,0.0,1.0
1560,0.333333,1.0,0.500000,0.02250,1.0,0.0,1.0
2035,0.166667,1.0,0.289474,0.01500,0.0,1.0,1.0
2394,0.076923,1.0,0.052632,0.30250,0.0,1.0,1.0
5451,0.115385,1.0,0.210526,0.03250,0.0,0.0,1.0
